In [1]:
import pandas as pd

# Create the dataset
data = {
    "weather": ["sunny", "sunny", "overcast", "rain", "rain", "rain", "overcast",
                "sunny", "sunny", "rain", "sunny", "overcast", "overcast", "rain"],
    "temp": ["hot", "hot", "hot", "mild", "cool", "cool", "cool",
             "mild", "cool", "mild", "mild", "mild", "hot", "mild"],
    "humidity": ["high", "high", "high", "high", "normal", "normal", "normal",
                 "high", "normal", "normal", "normal", "high", "normal", "high"],
    "wind": ["weak", "strong", "weak", "weak", "weak", "strong", "strong",
             "weak", "weak", "weak", "strong", "strong", "weak", "strong"],
    "answer": ["no", "no", "yes", "yes", "yes", "no", "yes",
               "no", "yes", "yes", "yes", "yes", "yes", "no"]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("3-dataset.csv", index=False)


In [2]:
import pandas as pd
import math
import numpy as np

# Load dataset
data = pd.read_csv("3.csv")

# Extract features (excluding the target)
features = [feat for feat in data.columns if feat != "answer"]

# Define Node class
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

# Entropy function
def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["answer"] == "yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    p = pos / (pos + neg)
    n = neg / (pos + neg)
    return -(p * math.log(p, 2) + n * math.log(n, 2))

# Information Gain function
def info_gain(examples, attr):
    uniq_vals = np.unique(examples[attr])
    gain = entropy(examples)
    for val in uniq_vals:
        subdata = examples[examples[attr] == val]
        gain -= (len(subdata) / len(examples)) * entropy(subdata)
    return gain

# ID3 algorithm
def ID3(examples, attrs):
    root = Node()
    max_gain = -1
    max_feat = ""

    for feature in attrs:
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature

    root.value = max_feat
    uniq_vals = np.unique(examples[max_feat])

    for val in uniq_vals:
        subdata = examples[examples[max_feat] == val]

        if entropy(subdata) == 0.0:
            leaf = Node()
            leaf.isLeaf = True
            leaf.value = val
            leaf.pred = subdata["answer"].iloc[0]
            root.children.append(leaf)
        else:
            child_node = Node()
            child_node.value = val
            new_attrs = [a for a in attrs if a != max_feat]
            child = ID3(subdata, new_attrs)
            child_node.children.append(child)
            root.children.append(child_node)
    return root

# Print Tree function
def printTree(root: Node, depth=0):
    if root.isLeaf:
        print("\t" * depth + f"{root.value} -> {root.pred}")
    else:
        print("\t" * depth + f"[{root.value}]")
        for child in root.children:
            printTree(child, depth + 1)

# Run algorithm
tree_root = ID3(data, features)
printTree(tree_root)


[weather]
	overcast -> yes
	[rain]
		[wind]
			strong -> no
			weak -> yes
	[sunny]
		[humidity]
			high -> no
			normal -> yes
